In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(6928123, 6)


,session,aid,ts,type,hour,day
0,12899779,59625,1661724000,0,79200,338400
1,12899780,1142000,1661724000,0,79200,338400
2,12899780,582732,1661724058,0,79258,338458
3,12899780,973453,1661724109,0,79309,338509
4,12899780,736515,1661724136,0,79336,338536


In [3]:
df.loc[df.aid==409236]

,session,aid,ts,type,hour,day
23800,12903836,409236,1661726102,0,81302,340502
50510,12909015,409236,1661733755,0,2555,348155
76927,12914149,409236,1661746059,0,14859,360459
79381,12914604,409236,1661746730,0,15530,361130
162566,12931131,409236,1661755665,0,24465,370065
...,...,...,...,...,...,...
6802008,14533201,409236,1662317906,0,68306,327506
6809380,14535398,409236,1662318066,0,68466,327666
6875550,14554705,409236,1662322327,0,72727,331927
6875551,14554705,409236,1662322422,0,72822,332022


In [4]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [5]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
0,12899779,59625,1661724000,0,79200,338400,<NA>,<NA>,0,0
1,12899780,1142000,1661724000,0,79200,338400,<NA>,559268,0,0
2,12899780,582732,1661724058,0,79258,338458,-559268,-390721,0,0
3,12899780,973453,1661724109,0,79309,338509,390721,236938,0,0
4,12899780,736515,1661724136,0,79336,338536,-236938,-405485,0,0


In [6]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [7]:
tmp.head()

aid
1059880    1.0
246407     1.0
1820683    1.0
1159625    1.0
895146     1.0
Name: n, dtype: float64

In [8]:
tmp.max(), tmp2.max()

(5.0, 57.0)

In [9]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

100

In [10]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
50328,1,0,0,0,-1.0,-1.0
50330,1,1,0,0,-1.0,-1.0
50335,5,5,0,0,-1.0,-1.0
50336,1,1,0,0,-1.0,-1.0
50339,2,3,0,0,-1.0,-1.0


In [11]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [12]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,4,4,44182.000000,303382.000000
3,0.123762,202,115,49449.787129,452792.361386
4,0.000000,8,5,60743.750000,319943.750000
6,0.000000,1,1,32317.000000,377917.000000
7,0.000000,1,1,38369.000000,38369.000000


In [13]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [14]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
1806078,-1.000000,-1.000000,-1.000000
1157963,0.690066,14641.736328,147252.875000
601007,0.000000,73.539108,73.539108
885662,0.423659,16392.638672,167094.796875
140995,0.000000,5291.987305,188574.062500


In [15]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [16]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [17]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,4,4,44182.000000,303382.000,1.000000,0.000000,19210.738281,74629.640625,3,4,0,0,-1.0,-1.000000
3,0.123762,202,115,49449.789062,452792.375,1.756522,0.372606,17274.923828,96579.421875,31,41,3,19,1.5,1.117647
4,0.000000,8,5,60743.750000,319943.750,1.600000,0.000000,14595.868164,128680.460938,7,5,0,0,-1.0,-1.000000
6,0.000000,1,1,32317.000000,377917.000,1.000000,-1.000000,-1.000000,-1.000000,1,1,0,0,-1.0,-1.000000
7,0.000000,1,1,38369.000000,38369.000,1.000000,-1.000000,-1.000000,-1.000000,0,0,0,0,-1.0,-1.000000


In [18]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [19]:
item_features.to_parquet('../../data/item_user_features/item10_LB.pqt')